# Deep Q Network
## Overview
In reinforcement learning, the agent interacts with environments to improve itself.

There are three types of data flow in RL training pipeline:
1. Agent to environment: `action` will be generated by agent and sent to environment
2. Environment to agent: `env.step` takes action, and returns a tuple of `(observation, reward, terminated, truncated, info)`
3. Agent-environment interaction to agent training: the data generated by interaction will be stored and sent to the learner of agent

In the following sections, we will set up (vectorised) environments, policy (with neural network), collector (with buffer), and trainer to successfully run th RL training and evaluation pipeline.

## Make an Environment
First of all, you have to make an environment for your agent to interact with.

In [2]:
import gymnasium as gym
import tianshou as ts

env = gym.make('CartPole-v0')

/Users/tanxiaoxuan/reinforcement_learning/RI_venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:523: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Setup Vectorized Environment
Tianshou supports vectorized environment for all algorithms. It provides four types of vectorized environment wrapper:
- `DummyVectorEnv`: the sequential version, using a single-thread for-loop
- `SubprocVectorEnv`: use python multiprocessing and pipe for concurrent execution
- `ShmemVectorEnv`: use share memory instead of pipe based on SubprocVectorEnv
- `RayVectorEnv`: use Ray for concurrent activities and is currently the only choice for parallel simulation in a cluster with multiple machines

Here we set up 10 environments in `train_envs` and 100 environments in `test_envs`.

In [3]:
train_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(100)])

/Users/tanxiaoxuan/reinforcement_learning/RI_venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:523: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Build the Network
Tianshou supports any user-defined PyTorch networks and optimizers. Yet, of course, the inputs and outputs must comply with Tianshou's API.

You can also use pre-defined MLP networks in `common`, `discrete` and `continuous`. The rules of self-defined networks are:
1. Input: observation `obs` (may be a `numpy.ndarray`, `torch.Tensor`, dict, or self-defined class), hidden state `state` (for RNN usage), and other information `info` provided by the environment
2. Output: some `logits`, the next hidden state `state`. The logits could be a tuple instead of a `torch.Tensor`, or some other useful variables or results during the policy forwarding procedure. It depends on how the policy class process the network ouput.

In [5]:
import torch
import numpy as np
from torch import nn

class Net(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape))
        )
    
    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float)
        batch = obs.shape[0]
        logits = self.model(obs.view(batch, -1))
        return logits, state
    
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape, action_shape)
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

## Setup Policy
We use the defined `net` and `optim` above, with extra policy hyper-parameters to define a policy. Here we define a DQN policy with target network:

In [7]:
policy = ts.policy.DQNPolicy(
    model=net,
    optim=optim,
    action_space=env.action_space,
    discount_factor=0.9,
    estimation_step=3,
    target_update_freq=320
)

## Setup Collector
The collector is a key concept in Tianshou. It allows the policy to interact with different types of environments conveniently. In each step, the collector will let the policy perform (at least) a specified number of steps or episodes and store the data in a replay buffer.

In [8]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)

The main function of collector is the collector function, which can be summarised in the following lines:

result = self.policy(self.data, last_state)  # the agent predicts the batch action from batch observation

act = to_numpy(result.act)

self.data.update(act=act)  # update the data with new action/policy

result = self.env.step(act, ready_env_ids)  # apply action to environment

obs_next, rew, done, info = result

self.data.update(obs_next=obs_next, rew=rew, done=done, info=info)  # update the data with new state/reward/done/info

## Train Policy with a Trainer
Tianshou provides `OnPolicyTrainer`, `OffPolicyTrainer` and `OfflineTrainer`. The trainer will automatically stop training when the policy reaches the stop condition `stop_fn` on test collector. Since DQN is an off-policy algorithm, we use the `OffPolicyTrainer` as follows:

In [13]:
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=10, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger=logger
).run()
print(f'Finished training! Use {result["duration"]}')

Epoch #1:   4%|4         | 450/10000 [00:03<01:23, 113.94it/s, env_step=450, len=200, n/ep=1, n/st=10, rew=200.00]            

Finished training! Use 4.83s


The meaning of each parameter is as follows:
- `max_epoch`: the maximum of epochs for training. the training process might be finished before reaching the `max_epoch`
- `step_per_epoch`: the number of environment step (transition) collected per epoch
- `step_per_collect`: the number of transition the collector would collect before the network update. For example, the code above means "collect 10 transitions and do one policy network update"
- `episode_per_test`: the number of episodes for one policy evaluation
- `batch_size`: the batch size of sample data, which is going to feed in the policy network
- `train_fn`: a function receives the current number of epoch and step index, and performs some operations at the beginning of training in this epoch. For example, the code above means "reset the epsilon to 0.1 in DQN before training"
- `test_fn`: a function receives the current number of epoch and step index, and performs some operations at the beginning of testing in this epoch. For example, the code above means "reset the epsilon to 0.05 in DQN before testing"
- `stop_fn`: a function receives the average undiscounted returns of the testing result, return a boolean which indicates whether reaching the goal
- `logger`: see below

The trainer supports `TensorBoard` for logging. It can be used as:

In [12]:
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
writer = SummaryWriter('log/dqn')
logger = TensorboardLogger(writer)

Pass the logger into the trainer, and the training result will be recorded into the TensorBoard.

In [17]:
print(result)

{'duration': '4.83s', 'train_time/model': '0.39s', 'test_step': 111047, 'test_episode': 600, 'test_time': '4.32s', 'test_speed': '25729.33 step/s', 'best_reward': 195.31, 'best_result': '195.31 ± 10.28', 'train_step': 450, 'train_episode': 5, 'train_time/collector': '0.12s', 'train_speed': '876.59 step/s'}


It shows that within 4.83 seconds, we finished training a DQN agent on CartPole. The mean returns over 100 consercutive episodes is 195.31.

## Save/Load Policy
Since the policy inherits the class `torch.nn.Module`, saving and loading the policy are exactly the same as a torch module:

In [18]:
torch.save(policy.state_dict(), 'dqn.pth')
policy.load_state_dict(torch.load('dqn.pth'))

<All keys matched successfully>

## Watch the Agent's Performance
`Collector` supports rendering. Here is the example of wtaching the agent's performance in 35 FPS:

In [19]:
policy.eval()
policy.set_eps(0.05)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/Users/tanxiaoxuan/reinforcement_learning/RI_venv/lib/python3.10/site-packages/tianshou/data/collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")
/Users/tanxiaoxuan/reinforcement_learning/RI_venv/lib/python3.10/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(


{'n/ep': 1,
 'n/st': 200,
 'rews': array([200.]),
 'lens': array([200]),
 'idxs': array([0]),
 'rew': 200.0,
 'len': 200.0,
 'rew_std': 0.0,
 'len_std': 0.0}

If you'd like to manually see the action generated by a well-trained agent:

assume obs is a single environment observation

action = policy(Batch(obs=np.array([obs]))).act[0]

## Train a Policy with Customised Codes
Tianshou supports user-defined training code.

In [25]:
# pre-collect at least 5000 transitions with random action before training
train_collector.collect(n_step=5000, random=True)

policy.set_eps(0.1)
for i in range(int(1e6)):  # total step
    collect_result = train_collector.collect(n_step=10)

    # once if the collected episodes' mean returns reach the threshold, or every 1000 steps, we test it on test_collector
    if collect_result['rews'].mean() >= env.spec.reward_threshold or i % 1000 == 0:
        policy.set_eps(0.05)
        result = test_collector.collect(n_episode=100)
        if result['rews'].mean() >= env.spec.reward_threshold:
            print(f'Finished training! Test mean returns: {result["rews"].mean()}')
            break
        else:
            # back to training eps
            policy.set_eps(0.1)
    
    # train policy with a sampled batch data from buffer
    losses = policy.update(64, train_collector.buffer)

/var/folders/tx/t6fnblk14z98ct88jxg69ctm0000gn/T/ipykernel_52827/42382888.py:9: RuntimeWarning: Mean of empty slice.
  if collect_result['rews'].mean() >= env.spec.reward_threshold or i % 1000 == 0:


Finished training! Test mean returns: 199.59
